<a href="https://colab.research.google.com/github/hossein-khalilian/Translator/blob/main/LSTM_Machine_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural machine translation with attention

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/app/LSTM_Machine_Translator")

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import io
import re
import time
import nltk
import numpy as np
import unicodedata
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
# Download the file
!wget http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip
path_to_file = 'deu.txt'

In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[0:2]]  for l in lines[:num_examples]]
  return zip(*word_pairs)

In [ ]:
en, sp = create_dataset(path_to_file, None)
print(en[0])
print(sp[0])

In [ ]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

In [ ]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer, inp_lang, targ_lang

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [ ]:
# Try experimenting with the size of that dataset
num_examples = 30000
# num_examples = None
input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [ ]:
# split train and test 
index = np.random.permutation(input_tensor.shape[0])
test_size = 0.2

input_tensor_val = input_tensor[index[:int(len(index)*test_size)]]
input_tensor_train = input_tensor[index[int(len(index)*(test_size)):]]
target_tensor_val = target_tensor[index[:int(len(index)*test_size)]]
target_tensor_train = target_tensor[index[int(len(index)*(test_size)):]]

inp_lang = np.array(inp_lang)
targ_lang = np.array(targ_lang)
input_lang_val = inp_lang[index[0:int(len(index)*test_size)]]
target_lang_val = targ_lang[index[0:int(len(index)*test_size)]]

print(input_tensor_train.shape)
print(input_tensor_val.shape)

### Create a tf.data dataset

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang_tokenizer.word_index)+1
vocab_tar_size = len(targ_lang_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset_val = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(input_tensor_val.shape[0], drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    # attention_weights = tf.reshape(attention_weights, (-1, ))
    # attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang_tokenizer.index_word[predicted_id] + ' '

    if targ_lang_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is Trainingfed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  return result

In [ ]:
unit_types = ['GRU', 'GRU_OneHot', 'LSTM', 'LSTM_MultiLayer']
unit_type = unit_types[0]
skip_training = True

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, unit_type='GRU'):
    super(Encoder, self).__init__()
    self.unit_type = unit_type
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.vocab_size = vocab_size
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.lstm = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform'
                                   )

  def call(self, x, hidden):
    if self.unit_type == 'GRU_OneHot':
        x = tf.one_hot(x,self.vocab_size)
    else:
        x = self.embedding(x)
    
    if self.unit_type == 'GRU' or self.unit_type == 'GRU_OneHot':
        output, state = self.gru(x, initial_state = hidden[0])
        state = [state,]

    elif self.unit_type == 'LSTM' or self.unit_type == 'LSTM_MultiLayer':
        output, state1, state2 = self.lstm(x, initial_state = hidden)
        state = [state1, state2]


    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, unit_type=unit_type)
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = [sample_hidden, sample_hidden]
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden[0], sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, unit_type='GRU'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.unit_type = unit_type
    self.dec_units = dec_units
    self.vocab_size = vocab_size
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.lstm = tf.keras.layers.LSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform'
                                   )
    self.lstm1 = tf.keras.layers.LSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform'
                                   )
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # hidden = tf.concat([hidden[0], hidden[1]], axis=-1)
    context_vector, attention_weights = self.attention(hidden[0], enc_output)

    x = self.embedding(x)

    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    if self.unit_type == 'GRU' or self.unit_type == 'GRU_OneHot':
        output, state = self.gru(x)
        state = [state,]

    elif self.unit_type == 'LSTM':
        output, state1, state2 = self.lstm(x)
        state = [state1, state2]

    elif self.unit_type == 'LSTM_MultiLayer':
        x1, state1, state2 = self.lstm(x)
        output, state1, state2 = self.lstm1(x1)
        state = [state1, state2]

    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, unit_type=unit_type)

sample_decoder_output, sample_hidden, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))

## Define the optimizer and the loss function

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [ ]:
checkpoint_dir = './training_checkpoints_German_20'+unit_type
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
if not skip_training:
  EPOCHS = 11
  training_loss = list()
  validation_loss = list()

  for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()

    enc_hidden = [enc_hidden, enc_hidden]

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      batch_loss = train_step(inp, targ, enc_hidden)
      total_loss += batch_loss

      if batch % 100 == 0:
        for (batch1, (inp1, targ1)) in enumerate(dataset_val.take(steps_per_epoch)):
          val_loss = train_step(inp, targ, enc_hidden)

        print('Epoch {} Batch {} training loss {:.4f} validation loss {:.4f}'\
              .format(epoch + 1,
              batch,
              batch_loss.numpy(),
              val_loss.numpy()))
        training_loss.append(batch_loss)
        validation_loss.append(val_loss)
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 10 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
plt.plot(training_loss)
plt.plot(validation_loss)
plt.title(unit_type + ' Loss')
plt.legend(['training_loss', 'validation_loss'])

# After training

## Restore the latest checkpoint and test

In [ ]:
# if skip_training:
# restoring the latest checkpoint in checkpoint_dir
checkpoint_dir = './training_checkpoints_German_'+unit_type
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
smoothie = nltk.translate.bleu_score.SmoothingFunction().method4

BLEU_score = 0
n = 1000
for i in range(n):
    s = input_lang_val[i].split('>')[1].split('<')[0][1:]
    result = translate(s).split('<')[0]
    target = target_lang_val[i].split('>')[1].split('<')[0][1:]
    BLEU_score += nltk.translate.bleu_score.sentence_bleu([target], result, smoothing_function=smoothie)
    # print(s)
    # print(target)
    # print(result)
    # print(BLEU_score)
    # print()

print('BLEU_score for ' + unit_type +' is: ', BLEU_score/n)

In [ ]:

print('BLEU_score for' + unit_type +' is: ', BLEU_score/n)


In [ ]:
for i in range(5):
    s = input_lang_val[i].split('>')[1].split('<')[0][1:]
    result = translate(s).split('<')[0]
    target = target_lang_val[i].split('>')[1].split('<')[0][1:]
    BLEU_score = nltk.translate.bleu_score.sentence_bleu([target], result, smoothing_function=smoothie)
    print(s)
    print(target)
    print(result)
    # print(BLEU_score)
    print()
